In [1]:
import pandas as pd

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-11 23:53:22,0.43945,0.0,0.0,0.0,2022-06-11,23:53:22,23
1,2022-06-11 23:54:22,0.48828,0.0,0.0,0.0,2022-06-11,23:54:22,23
2,2022-06-11 23:55:22,0.46387,0.0,0.0,0.0,2022-06-11,23:55:22,23
3,2022-06-11 23:56:22,0.46387,0.0,0.0,0.0,2022-06-11,23:56:22,23
4,2022-06-11 23:57:22,0.46387,0.0,0.0,0.0,2022-06-11,23:57:22,23


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
9571,2022-06-18 23:30:29,2022-06-18,23:30:29,23,0.17090,0.0,0.0,0.0
9572,2022-06-18 23:31:29,2022-06-18,23:31:29,23,0.26855,0.0,0.0,0.0
9573,2022-06-18 23:32:29,2022-06-18,23:32:29,23,0.09766,0.0,0.0,0.0
9574,2022-06-18 23:33:29,2022-06-18,23:33:29,23,0.24414,0.0,0.0,0.0
9575,2022-06-18 23:34:30,2022-06-18,23:34:30,23,0.17090,0.0,0.0,0.0


In [5]:
filter_daily_values = df[df['Date'] == '2022-06-18'][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
8169,2022-06-18,0,0.0
8170,2022-06-18,0,0.0
8171,2022-06-18,0,0.0
8172,2022-06-18,0,0.0
8173,2022-06-18,0,0.0
...,...,...,...
9571,2022-06-18,23,0.0
9572,2022-06-18,23,0.0
9573,2022-06-18,23,0.0
9574,2022-06-18,23,0.0


In [6]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-18,0,0.000000
1,2022-06-18,1,0.000000
2,2022-06-18,2,0.000000
3,2022-06-18,3,0.000000
4,2022-06-18,4,0.000000
5,2022-06-18,5,0.000000
6,2022-06-18,6,0.000000
7,2022-06-18,7,0.000000
8,2022-06-18,8,0.000000
9,2022-06-18,9,0.016607


In [7]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [8]:
weather_data = pd.read_csv('weather_data.csv', encoding= 'unicode_escape')
weather_data.drop(['RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText'], axis = 1, inplace = True)
weather_data

,SolarIrradiance (W/m2),weather status,Temp (°C),Hum (%),PreProbability (%),RainProbability (%),CloudCover (%)
0,0.0,Mostly cloudy,19.4,77,1,1,76
1,0.0,Mostly cloudy,18.7,78,4,4,76
2,0.0,Mostly cloudy,18.0,81,24,24,76
3,0.0,Mostly cloudy,17.4,85,24,24,76
4,0.0,Mostly cloudy,16.9,84,29,29,76
5,3.8,Mostly cloudy,16.4,82,43,43,76
6,12.2,Mostly cloudy,15.9,78,47,47,82
7,16.0,Showers,15.4,77,51,51,94
8,31.2,Mostly cloudy w/ t-storms,15.3,73,66,66,70
9,37.4,Intermittent clouds,14.4,73,49,49,70


In [9]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1

,Date,Hour,Power (KW),SolarIrradiance (W/m2),weather status,Temp (°C),Hum (%),PreProbability (%),RainProbability (%),CloudCover (%)
0,2022-06-18,0,0.000000,0.0,Mostly cloudy,19.4,77,1,1,76
1,2022-06-18,1,0.000000,0.0,Mostly cloudy,18.7,78,4,4,76
2,2022-06-18,2,0.000000,0.0,Mostly cloudy,18.0,81,24,24,76
3,2022-06-18,3,0.000000,0.0,Mostly cloudy,17.4,85,24,24,76
4,2022-06-18,4,0.000000,0.0,Mostly cloudy,16.9,84,29,29,76
5,2022-06-18,5,0.000000,3.8,Mostly cloudy,16.4,82,43,43,76
6,2022-06-18,6,0.000000,12.2,Mostly cloudy,15.9,78,47,47,82
7,2022-06-18,7,0.000000,16.0,Showers,15.4,77,51,51,94
8,2022-06-18,8,0.000000,31.2,Mostly cloudy w/ t-storms,15.3,73,66,66,70
9,2022-06-18,9,0.016607,37.4,Intermittent clouds,14.4,73,49,49,70
